In [ ]:
import cv2
import glob 
import numpy 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt 
from sklearn.model_selection import learning_curve
from skimage.feature import hog

# HOG

In [13]:
paths = glob.glob('training_set/**/*')

def load_images(paths, image_size):
    list_images = []
    list_labels = []
    for path in paths: 
        
        try:
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            #Pasar las imagenes al mismo tamaño 
            img = cv2.resize(img, image_size)
            #Flatten y dividir entre 255 
            # vector = img.flatten()/255.0
            features, image = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)
            list_images.append(features)
            #1 perro 
            if 'dogs' in path:
                list_labels.append(1)
            else: 
                #Gatos
                list_labels.append(0)
        except: 
            pass 
    return list_images, list_labels
        


In [14]:
image_size = (64,64)
list_images, list_labels = load_images(paths=paths, image_size=image_size)
X = list_images
y = list_labels

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [17]:
model = LogisticRegression()
model.fit(X_train, y_train)

/Users/joseandrescarvajalbautista/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
y_pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_pred_train, y_train)
print('accuracy_score:', accuracy_train)

accuracy_score: 0.8027795128044972


In [19]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print('accuracy score', accuracy)

accuracy score 0.7039350405996252


## Busqueda de hiperparametros


In [36]:
from sklearn.model_selection import GridSearchCV
import numpy as np 
parameters = {
    'C': np.arange(0.1, 1, 0.1),
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'max_iter': np.arange(100, 1000, 100),
    'solver': ['liblinear']
}

model = LogisticRegression()
grid = GridSearchCV(model, parameters, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_params_)

/Users/joseandrescarvajalbautista/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
810 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/joseandrescarvajalbautista/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/joseandrescarvajalbautista/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joseandres

{'C': 0.2, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'}


In [37]:
y_pred_train = grid.predict(X_train)
accuracy_train = accuracy_score(y_pred_train, y_train)
print('accuracy_score:', accuracy_train)

accuracy_score: 0.7484384759525297


In [38]:
y_pred = grid.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print('accuracy score', accuracy)

accuracy score 0.7058088694565896


# Maquinas de soporte vectorial

In [39]:
from sklearn.svm import SVC



X = list_images
y = list_labels

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


model = SVC()
model.fit(X_train, y_train)


SVC()

In [45]:
model = SVC(kernel='rbf', C=10, gamma=0.01)
model.fit(X_train, y_train)

SVC(C=10, gamma=0.01)

In [46]:
y_pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_pred_train, y_train)
print('accuracy_score:', accuracy_train)

accuracy_score: 0.8852279825109307


In [47]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print('accuracy score', accuracy)

accuracy score 0.7301686445971268


In [48]:
parameters = {
    'C': np.arange(0.1, 0.5, 0.1),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

In [ ]:
model = GridSearchCV(SVC(), parameters, cv=5, scoring='accuracy', n_jobs=-1)
model.fit(X_train, y_train)
print(model.best_params_)
y_pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_pred_train, y_train)
print('train accuracy_score:', accuracy_train)    
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print('test accuracy score', accuracy)